# 第8章 深度学习 

- 对于绝大多数问题，都可以期待通过加深网络来提高性能
- 在最近的图像识别大赛ILSVRC中，基于深度学习的方法独占鳌头，使用的网络也在深化
- VGG、GoogleNet、ResNet等是几个著名的网络
- 基于GPU、分布式学习、位数精度的缩减，可以实现深度学习的高速化
- 深度学习不仅可以用于物体识别，还可以用于物体检查、图像分割
- 深度学习的应用包括图像标题的生成、图像的生成、强化学习等。最近，深度学习在自动驾驶上的应用也备受期待。

## 8.1 加深网路 

创建一个进行手写数字识别的深度CNN
输入-conv-relu-conv-relu-pool(*3)-Affine-ReLU-Dropout-Affine-Dropout-Softmax
这个卷积层有如下特点：
- 基于3*3的小型滤波器的卷积层
- 激活函数是ReLU
- 全连接层的后面使用Dropout层
- 基于Adam的最优化
- 使用He的初始值作为权重初始值

In [1]:
# %load deep_convnet.py
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """识别率为99%以上的高精度的ConvNet

    网络结构如下所示
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 初始化权重===========
        # 各层的神经元平均与前一层的几个神经元有连接（TODO:自动计算）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # 使用ReLU的情况下推荐的初始值
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 生成层===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]


In [2]:
# %load train_deepnet.py
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录而进行的设定
import numpy as np
import matplotlib.pyplot as plt
from mnist import load_mnist
from deep_convnet import DeepConvNet
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)    #x_train长度为60000，x_test长度为10000

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 保存参数
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

train loss:2.2965710803557386
=== epoch:1, train acc:0.111, test acc:0.1 ===
train loss:2.2980105708453094
train loss:2.26363318381936
train loss:2.2858915788913894
train loss:2.2835555978606608
train loss:2.268136892580566
train loss:2.2650452006772084
train loss:2.2821945294165498
train loss:2.2419515904349816
train loss:2.261610549614618
train loss:2.2352433683791673
train loss:2.2643039273741037
train loss:2.2727100602173165
train loss:2.2077377866884844
train loss:2.234419748852371
train loss:2.1989507938843857
train loss:2.168210401957887
train loss:2.1834008031866734
train loss:2.170959386928643
train loss:2.2330886479609418
train loss:2.2204550232826734
train loss:2.1246138375131665
train loss:2.185455989219102
train loss:2.120743128027156
train loss:2.122029809810164
train loss:2.116273735984411
train loss:2.085262405395451
train loss:2.0656611079320912
train loss:2.0580873432854347
train loss:2.0460421844803833
train loss:1.999621431717964
train loss:2.0566842625271566
train 

KeyboardInterrupt: 

运行25min 1 epoch

对于MNIST数据集，层不用特别深就获得了最高的识别精度，对于手写数字识别这样比较简单的任务，没有必要将网络的表现力提高到那么高的程度。而对于大规模的一般物体识别的情况，因为问题复杂，所以加深层对提高识别精度大有裨益。

进一步提高识别精度的方法有集成学习、学习率衰减，数据扩充（data augmentation）等

- 加深层的动机

加深神经网络的好处是减少神经网络的参数，扩大感受野。

并且通过叠加层，将ReLU等激活函数夹在卷积层中间，进一步提高了网络的表现力。

加深层的另一个好处是使学习更加高效。通过加深网络，可以分层次地分解需要学习的问题。因此，各层需要学习的问题就变成了更简单的问题。这样一来，只需用较少的学习数据就可以高效进行学习。


感受野（Receptive Field）的定义是卷积神经网络每一层输出的特征图（feature map）上的像素点在输入图片上映射的区域大小。

## 8.2 深度学习的小历史 

2012年举办的大规模图像识别大赛ILSVRC(IamgeNet Large Scale Visual Recognition Challenge)中，基于深度学习的方法（AlexNet）以压倒性的优势胜出，彻底颠覆了以往的图像识别方法。

2015年的ResNet将错误识别率降低到了3.5%，这个结果甚至超过了普通人的识别能力。

这些年深度学习取得了不斐的成绩，其中，VGG、GoogleNet、ResNet已广为人知。

- VGG

VGG是由卷积层和池化层构成的基础的CNN，它的特点在于将有权重的层叠加至16层（或19层），具备了深度，根据层的不同，优势也称为“VGG16”或“VGG19”。

VGG中基于3*3小型滤波器的卷积层的计算是连续进行的。

- GoogLeNet

GoogLeNet的特征是不仅在纵向上有深度，在横向上也有深度（广度）。在横向上有宽度，这称为“Inception结构”，Inception结构使用了多个大小不同的滤波器，最后再合并他们的结果。

在GoogLeNet中，使用了多个大小为$1*1$的滤波器的卷积层，由于减少参数和实现高速化处理。

- ResNet

在深度学习中，过度加深层的话，很多情况下学习不能顺利进行，导致最终性能不佳。在ResNet中，导入了“快捷结构”，导入这个快捷结构后，就可以随着层的加深而不断提高性能了。

快捷结构横跨了输入数据的卷积层，将输入合计到输出，通过快捷结构，之前因加深层而导致的梯度消失问题有望得到缓解。

## 8.3 深度学习的高速化

在AlexNet中，大多数时间都被耗费在卷积层上，卷积层的处理时间加起来占GPU整体的95%,占CPU整体的89%，如何高速、高效地进行卷积层的运算是深度学习的一大课题。卷积层中进行的运算可以追溯至乘积累加运算。

因此，深度学习的主要课题就变成了如何高速、高效地进行乘积累加运算。

- 基于GPU的高速化

深度学习中需要进行大量的乘积累加运算（或者大型矩阵的乘积运算），这种大量的并行运算正是GPU所擅长的，反过来，CPU比较擅长连续的、复杂的计算。

通过im2col可以将卷积层进行的运算转换为大型矩阵的乘积，这个im2col的方式的实现对GPU来说是非常方便的实现，GPU更擅长计算大规模的汇总好的数据，通过基于im2col以大型矩阵的乘积的方式汇总计算，更容易发挥出GPU的能力。

- 分布式学习

为了进一步提高深度学习所需的计算的速度，可以考虑在多个GPU或者多台机器上进行分布式计算。

“如何进行分布式计算”是一个非常难的课题。它包含了机器之间的通信、数据的同步等多个无法轻易解决的问题，可以将这些难题都交给tensorflow这个框架。

- 位数缩减

计算机中表示小数时，有32位单精度浮点数和64位双警服浮点数等格式。在深度学习中，即使是16位的半精度浮点数，也可以顺利进行学习。

Numpy中提供了16位的半精度浮点数，只是16位类型的存储，运算本身不用16位进行，即使用Numpy的半精度浮点数，识别精度也不会下降。

In [7]:
# %load half_float_network.py
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录而进行的设定
import numpy as np
import matplotlib.pyplot as plt
from deep_convnet import DeepConvNet
from mnist import load_mnist


(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
network.load_params("deep_convnet_params.pkl")

sampled = 10000 # 为了实现高速化
x_test = x_test[:sampled]
t_test = t_test[:sampled]

print("caluculate accuracy (float64) ... ")
print(network.accuracy(x_test, t_test))

# 转换为float16型
x_test = x_test.astype(np.float16)
for param in network.params.values():
    param[...] = param.astype(np.float16)

print("caluculate accuracy (float16) ... ")
print(network.accuracy(x_test, t_test))

caluculate accuracy (float64) ... 
0.9935
caluculate accuracy (float16) ... 
0.9935


## 8.4 深度学习的应用案例 

深度学习不局限于物体识别，在图像、语音、自然语言等各个不同的领域，深度学习都展现了优异的性能。

- 物体检测

物体检测是从图像上确定物体的位置，并进行分类的问题。
对于物体检测问题，有多个基于CNN的方法，其中有一个R-CNN的方法。

R-CNN的处理流：1.Inuput image 2.Extract region proposals 
3. Compute CNN festures 4. Classify regions

- 图像分割

图像分割是指在像素水平上对图像进行分类。

有人提出一个FCN的方法（Fully Convolutional Network）,该方法通过一次forward处理，对所有像素进行分类，FCN在最后导入了扩大空间大小的处理，可将变小的中间数据扩大到和输入图像一样的大小。

- 图像标题的生成

一个基于深度学习生成图像标题的代表性方法是被称为NIC（Neural Image Caption）的模型，NIC由深层的CNN和处理自然语言的RNN构成，RNN具有循环连接的网络，经常被用于自然语言、时间序列数据等连续性的数据上。

基于NIC,可以生成惊人的高精度的图像标题，我们将组合图像和自然语言等多种信息进行的处理称为多模态处理。

## 8.5 深度学习的未来 

- 图像风格变换

输入两个图像后，会生成一个新的图像，两个输入图像中，一个称为“内容图像”，一个称为“风格图像”。

- 图像的生成

基于DBSCAN可以生成以假乱真的图像，使用大量图像训练这个模型，学习结束后，使用这个模型就可以生成新的图像。

DBSCAN中使用了深度学习，其技术要点是使用了Generator(生成者)和Discriminator(识别者)者两个神经网络。前者生成近似真品的图像，后者判断它是不是真的图像。像这样，通过让两者以竞争的方式去学习，Generator会学习到更精妙的图像作假技术，Discrinator会成为为能以更高精度辨别真假的鉴定师。

- 自动驾驶

自动驾驶需要结合多种技术力量来实现，比如路线计划技术、照相机或激光传感技术等，在这些技术中，正确识别时刻变化的环境是非常困难的。

基于CNN的神经网络SegNet，可以高精度地识别行驶环境。

- 强化学习

让计算机在摸索实践中自主学习，这称为强化学习。

强化学习的基本框架是，代理根据环境选择行动，然后通过这个行动改变环境，根据环境的变化，代理获得某种报酬。强化学习的目的是，决定代理的行动方针，以获得更好地报酬。

在深度学习中，有一个叫做Deep Q Network(DQN)的方法，这个方法基于被称为Q学习的强化学习算法，为了确定最合适的行动，需要确定一个被称为最优行动价值函数的函数。

在DQN的研究中，已有让电子游戏自动学习，实现超过人类水平操作的例子。

人工智能AlphaGo击败围棋冠军的新闻受到了广泛关注，这个AlphaGo技术的内部也使用了深度学习和强化学习。